# Worms speech

TODO Explain what this notebook is about and why it exists.


## Data preprocessing


In [26]:
import IPython.display as ipd

In [1]:
from pathlib import Path

import pandas as pd

raw = Path("data.txt").read_text()
rows = pd.Series(raw.splitlines())
rows.sample(n=10)

1889    *If you think that's painful, check out my 55-...
1137                                    * Lâche! (Coward)
2551                                      *Oh no! (Panic)
919                           Stereotypical French accent
704                *Are you new to the Darkside? (Stupid)
549                              * Loser! (Homing Weapon)
2574    *Shoot, and ask questions later. (Preparing at...
1665                                          *Uh. Hello.
2555                                *Mother! (Fall/Drown)
1867                               * What the? (What the)
dtype: object

In [2]:
rows.head(n=20)

0     '''FSpeech'''/'''Voices'''/'''Speechbanks''' (...
1                                                      
2                                      == Speech Key ==
3        * Representation - When a worm start his turn.
4     *Hello / Wait - Plays when no controls have be...
5     *Come On Then - Plays when [[Turn Time]] is do...
6     *Hurry - Plays when [[Turn Time]] is down to 5...
7     *Watch This - Plays while a variable power wea...
8     *Panic - One of several voice files that play ...
9                                            **Run Away
10                                     **Nooo - Falling
11    **Grenade - Panic file specifically for enemy ...
12    **Take Cover - When an enemy calls an [[Air St...
13    *Stupid / Oi Nutter- Plays when a enemy worm d...
14    *Traitor - Plays when a worm damages teammate(...
15             *Hurt - Enemy Worm after being attacked.
16                    *Missed - Enemy missed an attack.
17                               *Oops - Attack 

In [3]:
rows.tail(n=20)

3005    *Langweilig. / Das war alles? (No attacks duri...
3006                          *Bye bye. / Oh nein. (Dies)
3007                               *Gewinner! (Team wins)
3008          *Zahnräder in Bewegung! (Crafting a weapon)
3009    *Spezialapparat eingesetzt! (Activating specia...
3010                                                     
3011                                           ==Trivia==
3012    *One of Soul Man's lines, "Like a stacks machi...
3013    *The quote "Say hello to my little friend!" fr...
3014    *The phrases "Shoryuken" and "Hadouken" are re...
3015    *In ''Worms 2: Armageddon'', the phrase "Fire ...
3016    *The phrase from the Horror soundbank "I'll te...
3017    *In Worms HD for the PS3, some speech banks ha...
3018                       **Double Oh Seven is named 17.
3019                      **Poor Rapper is called Rapper.
3020                        **Snitch is named Backstreet.
3021    *Worms saying the word "fatality" is likely a ...
3022    *In [[

In [4]:
# TODO Make less hacky :)
df = rows[20:-13].str.extract(r"\*(.+)\((.*)\)").dropna()
df.columns = ["speech_line", "speech_key"]
df.sample(n=10)


,speech_line,speech_key
949,"Hon, hon, hon... Let's do this!",Orders
2957,We just wanna be left alone,Leave Me Alone
585,Fatality!,Fatality
1642,Every worm sees the light at the end of the tu...,Revenge
984,Ohé!,Hello
2573,It had t'be like this...,First Blood
1727,"Oops, troops.",Oops
1552,Get doon!,Take cover
2415,Is this really the best you have to offer?,Missed
117,It's the gravity I can't get used to.,Boring


In [5]:
# TODO Filter out speech keys with only one occurence.
df.groupby("speech_key").count().nlargest(25, columns="speech_line")

,speech_line
speech_key,
Death,56
Hurt,55
Coward,54
Hurry,51
Fatality,50
Panic,50
Stupid,49
Revenge,46
Boring,44


In [6]:
rows = df.rename(columns={"speech_key": "input", "speech_line": "output"}).to_dict(
    orient="records"
)

# TODO Send semistructured data directly instead of a string.
speechbank = ""
for d in rows:
    speechbank += f"input: {d['input']}\noutput: {d['output']}\n\n"

## Run text generation model to get speech lines


In [20]:
import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project="genai-nordics23sto-2304", location="us-central1")

parameters = {"temperature": 0.9, "max_output_tokens": 64, "top_p": 0.8, "top_k": 40}

model = TextGenerationModel.from_pretrained("text-bison")

# TODO Prepend personality.
worm_personality = """

"""

prompt = speechbank[:1000]

speech_key = "Missed"  # TODO Choice of speech key should be easier.

prompt += f"input: {speech_key}\noutput:"
text_response = model.predict(prompt, **parameters)

if text_response.text != "":
    print(f"{speech_key}: {text_response.text}")
else:
    print("No output :(")


Missed: A second chance is a rare opportunity! 


## Run text-to-speech (TTS) model to get audio

Synthesize speech from the input string of text.


In [27]:
from google.cloud import texttospeech

client = texttospeech.TextToSpeechClient()

input_text = texttospeech.SynthesisInput(text=text_response.text)

# Note: the voice can also be specified by name.
# Names of voices can be retrieved with client.list_voices().
voice = texttospeech.VoiceSelectionParams(
    language_code="en-US",
    name="en-US-Studio-M",
)

audio_config = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.LINEAR16, speaking_rate=1.0
)

audio_response = client.synthesize_speech(
    request={"input": input_text, "voice": voice, "audio_config": audio_config}
)

ipd.display(text_response.text, ipd.Audio(audio_response.audio_content, autoplay=True))

'A second chance is a rare opportunity! '

## Pitch-shifted audio for more Worms


In [30]:
import io

import librosa as lr

sample_rate = 16000

with io.BytesIO(audio_response.audio_content) as f:
    waveform, _ = lr.load(f, sr=sample_rate)
waveform = lr.effects.pitch_shift(waveform, sr=sample_rate, n_steps=12)

ipd.display(text_response.text, ipd.Audio(waveform, rate=sample_rate, autoplay=True))

'A second chance is a rare opportunity! '